--- Day 13: Shuttle Search ---

Your ferry can make it safely to a nearby port, but it won't get much further. When you call to book another ship, you discover that no ships embark from that port to your vacation island. You'll need to get from the port to the nearest airport.

Fortunately, a shuttle bus service is available to bring you from the sea port to the airport! Each bus has an ID number that also indicates how often the bus leaves for the airport.

Bus schedules are defined based on a timestamp that measures the number of minutes since some fixed reference point in the past. At timestamp 0, every bus simultaneously departed from the sea port. After that, each bus travels to the airport, then various other locations, and finally returns to the sea port to repeat its journey forever.

The time this loop takes a particular bus is also its ID number: the bus with ID 5 departs from the sea port at timestamps 0, 5, 10, 15, and so on. The bus with ID 11 departs at 0, 11, 22, 33, and so on. If you are there when the bus departs, you can ride that bus to the airport!

Your notes (your puzzle input) consist of two lines. The first line is your estimate of the earliest timestamp you could depart on a bus. The second line lists the bus IDs that are in service according to the shuttle company; entries that show x must be out of service, so you decide to ignore them.

To save time once you arrive, your goal is to figure out the earliest bus you can take to the airport. (There will be exactly one such bus.)

For example, suppose you have the following notes:

```
939
7,13,x,x,59,x,31,19
```

Here, the earliest timestamp you could depart is 939, and the bus IDs in service are 7, 13, 59, 31, and 19. Near timestamp 939, these bus IDs depart at the times marked D:

```
time   bus 7   bus 13  bus 59  bus 31  bus 19
929      .       .       .       .       .
930      .       .       .       D       .
931      D       .       .       .       D
932      .       .       .       .       .
933      .       .       .       .       .
934      .       .       .       .       .
935      .       .       .       .       .
936      .       D       .       .       .
937      .       .       .       .       .
938      D       .       .       .       .
939      .       .       .       .       .
940      .       .       .       .       .
941      .       .       .       .       .
942      .       .       .       .       .
943      .       .       .       .       .
944      .       .       D       .       .
945      D       .       .       .       .
946      .       .       .       .       .
947      .       .       .       .       .
948      .       .       .       .       .
949      .       D       .       .       .
```

The earliest bus you could take is bus ID 59. It doesn't depart until timestamp 944, so you would need to wait 944 - 939 = 5 minutes before it departs. Multiplying the bus ID by the number of minutes you'd need to wait gives 295.

What is the ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?


In [1]:
#!fsharp
let testInput = """939
7,13,x,x,59,x,31,19"""

let taskInput = """1000340
13,x,x,x,x,x,x,37,x,x,x,x,x,401,x,x,x,x,x,x,x,x,x,x,x,x,x,17,x,x,x,x,19,x,x,x,23,x,x,x,x,x,29,x,613,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,41"""

In [1]:
#!fsharp
type Specification = { timestamp: int; buses: int[] }

let parse (input:string) = 
    let parts = input.Replace("\r", "").Split("\n");
    let timestamp = int parts.[0]
    let buses = 
        parts.[1].Split(",") 
        |> Array.filter ( fun f -> f <> "x")
        |> Array.map int
    { timestamp = timestamp; buses = buses }

testInput |> parse

timestamp,buses
939,"[ 7, 13, 59, 31, 19 ]"


In [1]:
#!fsharp
let firstDepartures (specs: Specification) =
    specs.buses
    |> Array.map
        (fun id -> 
            let mul = int( ceil( double specs.timestamp / double id ) )
            (id, mul * id, mul * id - specs.timestamp)
        )

testInput |> parse |> firstDepartures

index,Item1,Item2,Item3
0,7,945,6
1,13,949,10
2,59,944,5
3,31,961,22
4,19,950,11


In [1]:
#!fsharp
let findEarliest departures =
    departures
    |> Array.minBy (fun (id, first, wait) -> first)

testInput |> parse |> firstDepartures |> findEarliest

Item1,Item2,Item3
59,944,5


In [1]:
#!fsharp
testInput |> parse |> firstDepartures |> findEarliest |> ( fun (id, first, wait) -> id * wait )

295

In [1]:
#!fsharp
taskInput |> parse |> firstDepartures |> findEarliest |> (fun (id, first, wait) -> id * wait )

136

The shuttle company is running a contest: one gold coin for anyone that can find the earliest timestamp such that the first bus ID departs at that time and each subsequent listed bus ID departs at that subsequent minute. (The first line in your input is no longer relevant.)

For example, suppose you have the same list of bus IDs as above:

7,13,x,x,59,x,31,19

An x in the schedule means there are no constraints on what bus IDs must depart at that time.

This means you are looking for the earliest timestamp (called t) such that:

*    Bus ID 7 departs at timestamp t.
*    Bus ID 13 departs one minute after timestamp t.
*    There are no requirements or restrictions on departures at two or three minutes after timestamp t.
*    Bus ID 59 departs four minutes after timestamp t.
*    There are no requirements or restrictions on departures at five minutes after timestamp t.
*    Bus ID 31 departs six minutes after timestamp t.
*    Bus ID 19 departs seven minutes after timestamp t.

The only bus departures that matter are the listed bus IDs at their specific offsets from t. Those bus IDs can depart at other times, and other bus IDs can depart at those times. For example, in the list above, because bus ID 19 must depart seven minutes after the timestamp at which bus ID 7 departs, bus ID 7 will always also be departing with bus ID 19 at seven minutes after timestamp t.

In this example, the earliest timestamp at which this occurs is 1068781:

```
time     bus 7   bus 13  bus 59  bus 31  bus 19
1068773    .       .       .       .       .
1068774    D       .       .       .       .
1068775    .       .       .       .       .
1068776    .       .       .       .       .
1068777    .       .       .       .       .
1068778    .       .       .       .       .
1068779    .       .       .       .       .
1068780    .       .       .       .       .
1068781    D       .       .       .       .
1068782    .       D       .       .       .
1068783    .       .       .       .       .
1068784    .       .       .       .       .
1068785    .       .       D       .       .
1068786    .       .       .       .       .
1068787    .       .       .       D       .
1068788    D       .       .       .       D
1068789    .       .       .       .       .
1068790    .       .       .       .       .
1068791    .       .       .       .       .
1068792    .       .       .       .       .
1068793    .       .       .       .       .
1068794    .       .       .       .       .
1068795    D       D       .       .       .
1068796    .       .       .       .       .
1068797    .       .       .       .       .
```

In the above example, bus ID 7 departs at timestamp 1068788 (seven minutes after t). This is fine; the only requirement on that minute is that bus ID 19 departs then, and it does.

Here are some other examples:

*    The earliest timestamp that matches the list 17,x,13,19 is 3417.
*    67,7,59,61 first occurs at timestamp 754018.
*    67,x,7,59,61 first occurs at timestamp 779210.
*    67,7,x,59,61 first occurs at timestamp 1261476.
*    1789,37,47,1889 first occurs at timestamp 1202161486.

However, with so many bus IDs in your list, surely the actual earliest timestamp will be larger than 100000000000000!

What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

In [1]:
#!fsharp
let parse2 (input: string) =
    input.Replace("\r", "").Split("\n").[1].Split(",")
    |> Array.mapi (fun i id -> (i, id) )
    |> Array.filter ( fun (i, id) -> id <> "x")
    |> Array.map ( fun (i, id) -> (int64 i, int64 id))
    
parse2 testInput

index,Item1,Item2
0,0,7
1,1,13
2,4,59
3,6,31
4,7,19


In [1]:
#!fsharp
let findGoal (buses: (int64 * int64) array) =
    let rec step timestamp =
        let valid = Array.forall ( fun (offset, id) ->
                                    (timestamp + offset) % id = 0L
                                 ) buses
        if valid = true then 
            timestamp
        else
            step (timestamp + (snd buses.[0]))
    step (snd buses.[0])

testInput |> parse2 |> findGoal

1068781

In [1]:
#!fsharp
let rec gcd a b =
    if a = 0L then
        b
    else
        gcd (b % a) a

let rec lcm a b =
    let div = gcd a b
    a * (b / div)

lcm 15L 3L

15

In [1]:
#!fsharp
let testInput2 = """0
17,x,13,19"""

let testInput3 = """0
67,7,59,61"""

let testInput4 = """0
67,x,7,59,61"""

let testInput5 = """0
67,7,x,59,61"""

let testInput6 = """0
1789,37,47,1889"""

In [1]:
#!fsharp
let testInputA = """0
17,x,13"""

testInputA |> parse2 |> findGoal

102

In [1]:
#!fsharp
let findGoal2 (buses: (int64 * int64) array) =
    let rec inner (timestamp: int64) (multiply: int64) (buses: (int64 * int64) list) =
        match buses with
        | [] -> timestamp
        | (offs,id)::xr ->
            let mutable newTimestamp = timestamp
            while (newTimestamp + offs ) % id <> 0L do 
                newTimestamp <- newTimestamp + multiply
            inner newTimestamp (lcm multiply id) xr

    let asList = List.ofArray buses
    inner (snd buses.[0]) 1L asList
    
testInput2 |> parse2 |> findGoal2

3417

In [1]:
#!fsharp
[
testInput2 |> parse2 |> findGoal2
testInput3 |> parse2 |> findGoal2
testInput4 |> parse2 |> findGoal2
testInput5 |> parse2 |> findGoal2
testInput6 |> parse2 |> findGoal2
]

index,value
0,3417
1,754018
2,779210
3,1261476
4,1202161486


In [1]:
#!fsharp
taskInput |> parse2 |> findGoal2

305068317272992

In [1]:
#!fsharp
let rec gcdExt a b =
    if a = 0L then
        (b,0L,1L)
    else
        let res, x1, y1 = gcdExt (b%a) a
        (res, y1 - (b / a) * x1, x1)

let modInverse b m =
    let g, x, y = gcdExt b m

    if g <> 1L then
        -1L
    else
        (x % m + m) % m

let modDiv a b m =
    let a2 = a % m
    let inv = modInverse b m
    if inv = -1L then
        failwith "Division not defined" 
    else
        (inv * a2) % m
//gcdExt 6L 4L
modDiv 1L 3L 7L

5

In [1]:
#!fsharp
let findGoal3 (buses: (int64 * int64) array) =
    let rec inner (timestamp: int64) (multiply: int64) (buses: (int64 * int64) list) =
        match buses with
        | [] -> timestamp
        | (offs,id)::xr ->
            // we want to be sure that   (delta + offs + timestamp) % id = 0, where delta is multiply of "multiply"
            // we turn it into delta  = 0 - offs - timestamp      in modulo id
            // and solve for  X * multiply = 0 - offs - timestamp  in modulo id 
            let numerator = (id + (-offs - timestamp) % id) % id
            let div = modDiv numerator multiply id
            let newTimestamp = timestamp + (div * multiply)
            inner newTimestamp (lcm multiply id) xr

    let asList = List.ofArray buses
    inner (snd buses.[0]) 1L asList

testInput2 |> parse2 |> findGoal3
//(13 + (-2 - 17) % 13) % 13

3417

In [1]:
#!fsharp
[
testInput2 |> parse2 |> findGoal3
testInput3 |> parse2 |> findGoal3
testInput4 |> parse2 |> findGoal3
testInput5 |> parse2 |> findGoal3
testInput6 |> parse2 |> findGoal3
]

index,value
0,3417
1,754018
2,779210
3,1261476
4,1202161486


In [1]:
#!fsharp
taskInput |> parse2 |> findGoal3

305068317272992